In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('database_24_25.csv')
df.head()

,Player,Tm,Opp,Res,MP,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,Data
0,Jayson Tatum,BOS,NYK,W,30.30,14,18,0.778,8,11,...,4,4,10,1,1,1,1,37,38.1,2024-10-22
1,Anthony Davis,LAL,MIN,W,37.58,11,23,0.478,1,3,...,13,16,4,1,3,1,1,36,34.0,2024-10-22
2,Derrick White,BOS,NYK,W,26.63,8,13,0.615,6,10,...,3,3,4,1,0,0,1,24,22.4,2024-10-22
3,Jrue Holiday,BOS,NYK,W,30.52,7,9,0.778,4,6,...,2,4,4,1,0,0,2,18,19.5,2024-10-22
4,Miles McBride,NYK,BOS,L,25.85,8,10,0.800,4,5,...,0,0,2,0,0,1,1,22,17.8,2024-10-22


In [7]:
df.describe()

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc
count,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000,3844.000000
mean,22.007867,3.740895,8.070760,0.424989,1.210978,3.388398,0.264357,1.605099,2.063476,0.420124,0.988033,3.001041,3.989074,2.351457,0.753642,0.470083,1.277575,1.807752,10.297867,8.281322
std,11.093446,3.240950,6.084169,0.261623,1.514429,3.146485,0.292502,2.267014,2.754057,0.435133,1.331327,2.683883,3.413613,2.568357,0.993827,0.842156,1.455277,1.490968,8.831583,7.755040
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.200000
25%,13.815000,1.000000,3.000000,0.273000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,2.200000
50%,23.090000,3.000000,7.000000,0.444000,1.000000,3.000000,0.222000,1.000000,1.000000,0.400000,1.000000,2.000000,3.000000,2.000000,0.000000,0.000000,1.000000,2.000000,8.000000,6.800000
75%,31.330000,6.000000,12.000000,0.571000,2.000000,5.000000,0.500000,2.000000,3.000000,1.000000,1.000000,4.000000,6.000000,3.000000,1.000000,1.000000,2.000000,3.000000,15.000000,12.825000
max,47.780000,21.000000,34.000000,1.000000,10.000000,18.000000,1.000000,17.000000,22.000000,1.000000,10.000000,15.000000,20.000000,16.000000,7.000000,9.000000,10.000000,6.000000,59.000000,54.200000


In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [20]:
df_curry = df[df['Player'] == 'Jayson Tatum']

In [21]:
df_curry.head()

,Player,Tm,Opp,Res,MP,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,Data
0,Jayson Tatum,BOS,NYK,W,30.30,14,18,0.778,8,11,...,4,4,10,1,1,1,1,37,38.1,2024-10-22
282,Jayson Tatum,BOS,WAS,W,28.65,9,20,0.450,3,11,...,9,11,6,1,0,1,1,25,21.3,2024-10-24
613,Jayson Tatum,BOS,DET,W,38.13,12,26,0.462,6,13,...,3,4,2,2,1,1,5,37,25.5,2024-10-26
1044,Jayson Tatum,BOS,MIL,W,36.42,6,16,0.375,1,8,...,8,8,4,2,1,3,5,15,9.1,2024-10-28
1269,Jayson Tatum,BOS,IND,L,43.37,10,25,0.400,5,18,...,7,8,4,4,0,3,3,37,27.7,2024-10-30
